# Cell Density Notebook

Ben Halligan

## Purpose

Count number of cells in given area for microscope dataset

## Images Parameters

- Size: 2000 x 2000 x 1

### TODO:

- [x] Connect to google drive
- [x] Load tiffs into dataframe
- [x] Test train split
- [x] General linear regression model
- [ ] Train
- [ ] Validate

### Models

#### VGG16 Model Description

- Based on [this](https://github.com/LeanManager/PyTorch_Image_Classifier/blob/master/Image_Classifier_Project.ipynb) project
- [VGG16 Model](https://neurohive.io/en/popular-networks/vgg16/)
- Basic classification model for multiple classes

#### Density Map CNN

- Creates heat maps of where obejects are
- Estimates how many objects are there
- [Descripition](https://towardsdatascience.com/objects-counting-by-estimating-a-density-map-with-convolutional-neural-networks-c01086f3b3ec)
- [Github](https://github.com/NeuroSYS-pl/objects_counting_dmap)
- Going to either annotate data, **OR** make something to find cells for counting


#### Other One implement later, maybe

- [Link](https://papers.nips.cc/paper/2010/file/fe73f687e5bc5280214e0486b273a5f9-Paper.pdf)

In [1]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

from typing import Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from skimage import(
    io, measure
)

# check for GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Current Device: {device}")

# Google Auth
# gauth = GoogleAuth()
# gauth.LocalWebserverAuth()

# drive = GoogleDrive(gauth)

PATH = 'data/INS1_BF/'

Current Device: cuda:0


In [2]:
file_list = os.listdir(PATH)
print(file_list)

['6000', '4000', '2000', '10000', '8000', '0']


In [3]:
# Create Data


data = datasets.ImageFolder(root='data/INS1_BF', transform=transforms.ToTensor())
print(data.classes)

In [6]:
# Load data with data loader

loader = DataLoader(data, batch_size= 64, shuffle=True)
dataiter = iter(loader)
images, labels = dataiter.next()
print(type(images))
print(images.shape)
print(labels.shape)

<class 'torch.Tensor'>
torch.Size([64, 3, 2000, 2000])
torch.Size([64])


In [12]:
# Test Train Split
train_size = int(0.8*len(data))
test_size = len(data) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(data, [train_size, test_size])

In [13]:
# Create test and train dataloaders

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4,
                                         shuffle=True, num_workers=2)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=4,
                                        shuffle=True, num_workers=2)